In [1]:
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob

# 
from scipy.io import loadmat
import scipy

# 

Autosaving every 180 seconds


In [ ]:
###################################
######## USE .mat DATASETS ########
###################################
# trx = {
#  'x': x,                          0
#  'y':y,                           1
#  'theta': self.angles[:,k],       2 
#  'a': self.axes[:,k,0],           3
#  'b': self.axes[:,k,1],           4
#  'nframes':nframes,               5
#  'firstframe':firstframe,         6
#  'endframe':endframe,             7
#  'off':off,                       8 
#  'id':id_,                        9
#  'x_mm':x_mm,                    10
#  'y_mm':y_mm,                    11
#  'theta_mm':theta_mm[:,k],       12
#  'a_mm':a_mm,                    13
#  'b_mm':b_mm,                    14
#  'sex':sex,                      15 
#  'dt':dt,                        16
#  'fps':self.fps,                 17
#  'timestamps': timestamps        18
# }

# 
fnames = np.loadtxt('/media/cat/1TB/dan/cohort1/slp.txt',
                   dtype='str')

# 
X4 = []
for k in range(4):
    X4.append([])

#
animal_ids = np.arange(4)
for fname in fnames:
    data = loadmat(fname)

    for animal_id in animal_ids:
        #print (data['trx'][0][0][animal_id][0][0][2].squeeze())
        data_animal = data['trx'][0][0][animal_id][0][0]

        x = data_animal[0]
        y = data_animal[1]
        angle = data_animal[2]

        locs = np.vstack((x, y, angle)).T
        #print (locs.shape)

        vel = np.sqrt((locs[1:,0]-locs[:-1,0])**2+
                      (locs[1:,1]-locs[:-1,1])**2
                     )
        
        # find locations where velocity > first initial large peak
        # first visualize where peak in movements start
        # width = 1
        # bins = np.arange(1,100,width)
        # y = np.histogram(vel,bins=bins)
        # plt.plot(y[1][:-1],y[0])
        idx = np.where(vel>=5)[0]

        # look for starts that are separated by at least X
        window = 25
        starts = []
        for k in range(1, idx.shape[0],1):
            if idx[k]-idx[k-1]>=window:
                starts.append(idx[k-1])

        starts = np.array(starts)
        #print (starts.shape)

        X = []
        for k in range(starts.shape[0]):
            temp = locs[starts[k] : starts[k]+window]
            
            if np.where(np.isnan(temp))[0].shape[0]==0:
                
                # featurize to velocity and angle
                if False:
                    wrong
                    temp = vel([1:,0]-temp[:-1,0])
                    #print (vel.shape, temp[1:,2].shape)
                    temp = np.vstack((vel, temp[1:,2]))
            
                X.append(temp)

        X = np.array(X)
        #print (X.shape)
        X4[animal_id].extend(X)
        
# 
for k in range(len(X4)):
    temp = X4[k]
    temp = np.array(temp)
    X4[k] = temp

print (X4[0].shape)

In [37]:
##############################################
########## FEATURIZE BEHAVIOR CHUNKS #########
##############################################
from sklearn import decomposition
import sklearn

fig = plt.figure()
X_all = []
n_events = []
for animal_id in animal_ids:
    X = X4[animal_id].copy()
    X = X.reshape(X.shape[0], -1)
    print (X.shape)
    X_all.append(X)
    n_events.append(X.shape[0])

#     
X_all = np.vstack(X_all)
print (X_all.shape)
X = sklearn.preprocessing.normalize(X_all)

#
if True:
    pca = decomposition.PCA(n_components=3)

    X_pca = pca.fit_transform(X_all)
    print (X_pca.shape)
    
if False:
    import umap
    umap = umap.UMAP(n_components=2,
                    init='random',
                    random_state=0)

    umap_ = umap.fit(X_all[::10])

    X_pca = umap_.transform(X_all)
        

print ("plotting: ", X_pca.shape)


print (n_events)
fig=plt.figure()
for k in range(4):
    ax = plt.subplot(2,2,k+1)
    start = np.int32(n_events[:k]).sum()
    end = np.int32(n_events[:k+1]).sum()
    print (start, end)
    plt.scatter(X_pca[start:end,0],
                X_pca[start:end,1],
               alpha=.1)

plt.show()


(11594, 48)
(13104, 48)
(13240, 48)
(12227, 48)
(50165, 48)
(50165, 3)
plotting:  (50165, 3)


In [38]:
##########################################
##########################################
##########################################




[11594, 13104, 13240, 12227]
0 11594
11594 24698
24698 37938
37938 50165


In [3]:
data = np.load('/media/cat/256GB/dan/slp/prediction_ExtraTreesRegressor_animalID_0_0_1.npy')
print (data.shape)
print (data[:10])

(2069710, 6, 2)
[[[ 0.0000000e+00  0.0000000e+00]
  [ 1.9968408e-15 -3.2624161e+01]
  [-7.1101522e-01 -5.7264263e+01]
  [-2.3980792e+00 -8.0996239e+01]
  [-3.7390034e+00 -1.0222711e+02]
  [-5.1375160e+00 -1.1838454e+02]]

 [[ 0.0000000e+00  0.0000000e+00]
  [ 1.9968408e-15 -3.2624161e+01]
  [-7.1101522e-01 -5.7264263e+01]
  [-2.3980792e+00 -8.0996239e+01]
  [-3.7390034e+00 -1.0222711e+02]
  [-5.1375160e+00 -1.1838454e+02]]

 [[ 0.0000000e+00  0.0000000e+00]
  [ 1.9968408e-15 -3.2624161e+01]
  [-7.1101522e-01 -5.7264263e+01]
  [-2.3980792e+00 -8.0996239e+01]
  [-3.7390034e+00 -1.0222711e+02]
  [-5.1375160e+00 -1.1838454e+02]]

 [[ 0.0000000e+00  0.0000000e+00]
  [ 1.9968408e-15 -3.2624161e+01]
  [-7.1101522e-01 -5.7264263e+01]
  [-2.3980792e+00 -8.0996239e+01]
  [-3.7390034e+00 -1.0222711e+02]
  [-5.1375160e+00 -1.1838454e+02]]

 [[ 0.0000000e+00  0.0000000e+00]
  [ 1.9968408e-15 -3.2624161e+01]
  [-7.1101522e-01 -5.7264263e+01]
  [-2.3980792e+00 -8.0996239e+01]
  [-3.7390034e+00 -1.022

In [74]:
feature_ids = np.array([0,5,6,7,8,9])
    
d = np.load('/media/cat/1TB/dan/cohort1/slp/animalID_0_alldata.npy')[:,feature_ids]
print (d.shape)

d_imp = np.load('/media/cat/1TB/dan/cohort1/slp/animalID_0_alldata_imputed.npy')
print (d_imp.shape)

idx_all = np.arange(d.shape[0])
ctr=0
sizes = (np.arange(1,7,1)*0.1)[::-1]

fig = plt.figure(figsize=(10,10))
while ctr<10:
#while True:
    idx = np.random.choice(idx_all, 1)[0]
    
    idx2 = np.where(np.isnan(d[idx][:,0])==False)[0]
    idx3 = np.where(np.isnan(d_imp[idx,:,0])==False)[0]

    if idx2.shape[0] != idx3.shape[0] and idx2.shape[0]>=5:
    #if idx2.shape[0]>=2:
        #idx3 = np.where(np.isnan(d_imp[idx,:,0])==False)[0]

        print (idx, idx2, idx3, sizes[idx2])
        print ("preimupted: ", d[idx])

        plt.subplot(2,5,ctr+1)
        ss = np.zeros((6))+np.nan
        ss[idx3] = sizes[idx3]
        plt.scatter(d_imp[idx,:,0],
                    d_imp[idx,:,1],
                    s=ss,
                    c='blue')

        ss = np.zeros((6))+np.nan
        ss[idx2] = sizes[idx2]
        print ("sizes: ", ss)
        plt.scatter(d[idx,:,0],
                    d[idx,:,1],
                    s=ss,
                   c='black')
        ctr+=1
        plt.title(str(idx)+ " " +str(idx2), fontsize=3, pad=0.8)
        plt.xticks([])
        plt.yticks([])
        plt.xlim(0,1280)
        plt.ylim(0,1280)

if True:
    plt.savefig('/home/cat/impute.png', dpi=300)
    plt.close()
else:
    plt.show()

(2069710, 6, 2)
(2069710, 6, 2)
40302 [0 1 2 3 4] [0 1 2 3 4 5] [0.6 0.5 0.4 0.3 0.2]
preimupted:  [[379.76889038 675.94378662]
 [339.96218872 676.4152832 ]
 [308.32745361 679.38964844]
 [287.98355103 694.94458008]
 [271.35760498 712.30834961]
 [         nan          nan]]
sizes:  [0.6 0.5 0.4 0.3 0.2 nan]
18536 [0 1 2 3 4] [0 1 2 3 4 5] [0.6 0.5 0.4 0.3 0.2]
preimupted:  [[364.61849976 639.13476562]
 [335.53079224 619.293396  ]
 [311.76940918 608.07891846]
 [280.07833862 608.21850586]
 [315.88220215 599.0145874 ]
 [         nan          nan]]
sizes:  [0.6 0.5 0.4 0.3 0.2 nan]
40296 [0 1 2 3 4] [0 1 2 3 4 5] [0.6 0.5 0.4 0.3 0.2]
preimupted:  [[379.59872437 675.94378662]
 [339.94113159 676.52996826]
 [308.1541748  679.48248291]
 [287.98355103 694.94458008]
 [271.35760498 712.30834961]
 [         nan          nan]]
sizes:  [0.6 0.5 0.4 0.3 0.2 nan]
18535 [0 1 2 3 4] [0 1 2 3 4 5] [0.6 0.5 0.4 0.3 0.2]
preimupted:  [[364.61849976 639.13476562]
 [335.53079224 619.293396  ]
 [311.76940918 